### Idee

**Questa sezione viene utilizzata per scrivere tutto ciò che ci viene in mente da poter implementare per rendere il progetto più interessante**

- Usare le colonne di ground truth (drink e smoke) come features vere e proprie per la classificazione di smoke e drink
- Provare a fare classificazione senza includere le colonne di ground truth (drink e smoke) come features
- Fare un modello che predica se il paziente mente oppure no (probabilmente si fare clustering)

In [86]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Download latest version
path = kagglehub.dataset_download("sooyoungher/smoking-drinking-dataset")
print("Path to dataset files:", path)

file_name = "smoking_driking_dataset_Ver01.csv"
file_path = f"{path}/{file_name}"
dataset = pd.read_csv(file_path)

In [ ]:
# Analisi iniziale del dataset
print("\nPrime righe del dataset:")
print(dataset.head())

print("\nDimensioni del dataset:")
print(dataset.shape)

print("\nTipi di dati:")
print(dataset.info())

print("\nStatistiche descrittive di base:")
print(dataset.describe())


### Prime considerazioni

I dati sembrano non contenere dati nulli o NaN, però si osserva facilmente che per alcune feature ci sono dei dati che sembrano essere irrealistici (Es.: per la feature **waistline** la media risulta essere _81_, il 4° quartile _87_ e il valore massimo _999_); quindi nonostante la non presenza di valori nulli, bisogna verificare se sono presenti valori mancanti codificati in altro modo.

In [ ]:
smoke = dataset["SMK_stat_type_cd"].value_counts()
drink = dataset["DRK_YN"].value_counts()

# Distribuzione SMK_stat_type_cd
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1) # Num righe, num colonne, posizione 
sns.barplot(x=smoke.index, y=smoke.values, palette="Blues_d")
plt.title("Distribuzione delle classi relative al fumo")
plt.xlabel("Tipo di fumatore")
plt.ylabel("Frequenza")

# Distribuzione DRK_YN
plt.subplot(1, 2, 2)
sns.barplot(x=drink.index, y=drink.values, palette="Greens_d")
plt.title("Distribuzione delle classi relative al bere")
plt.xlabel("Bevitore (Y/N)")
plt.ylabel("Frequenza")

plt.tight_layout()
plt.show()

In [ ]:
dataset_mod = dataset.drop(columns=["sex", "SMK_stat_type_cd", "DRK_YN"])

# Boxplot per le principali feature numeriche
plt.figure(figsize=(25, 20))
for i, col in enumerate(dataset_mod.columns):
    plt.subplot(5, 5, i + 1)
    sns.boxplot(y=dataset_mod[col], color="skyblue")
    plt.title(col)
    plt.tight_layout()
plt.show()

In [ ]:
# Verifica dei valori massimi e distanza dal secondo massimo e dalla media

for col in dataset_mod.columns:
    max = dataset_mod[col].max()
    max_count = (dataset_mod[col] == max).sum()
    second_max = dataset_mod[col][dataset_mod[col] < max].max()
    distance_max = max - second_max
    mean_value = dataset[col].mean()
    distance_mean = max - mean_value

    print(f"Colonna: {col}")
    print(f"    -Valore massimo: {max}")
    print(f"    -Occorrenze del massimo: {max_count}")
    print(f"    -Secondo massimo: {second_max}")
    print(f"    -Distanza tra massimo e secondo massimo: {distance_max}")
    print(f"    -Media: {mean_value}")
    print(f"    -Distanza tra massimo e media: {distance_mean}")
    print()

In [ ]:
# Determinazione del numero di valori "fuori scala" (outliers) per ogni feature

for col in dataset_mod.columns:
    Q1 = dataset_mod[col].quantile(0.25)
    Q3 = dataset_mod[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = dataset_mod[(dataset_mod[col] < lower_bound) | (dataset_mod[col] > upper_bound)]
    print(f"Colonna: {col}")
    print(f"    -Valori fuori scala: {len(outliers)}")
    print(f"    -Limiti: {lower_bound} - {upper_bound}")
    print()

### Considerazioni

Un'analisi più approfondita ha permesso di individuare la presenza di alcuni outlier in specifiche feature, in particolare confrontando il valore massimo presente nel dataset con il secondo valore massimo per ciascuna di esse; è importante sottolineare però che non tutte le feature mostrano grosse differenze tra il primo e il secondo massimo ed in molti casi sono presenti molti valori tra il massimo e la media.

Sono state fatte dunque delle brevi ricerche per capire quali valori fossero plausibili per ogni feature e quali no, è stato deciso di stabilire una soglia massima accettabile per alcune di queste; queste soglie sono state definite con l'obiettivo di preservare il maggior numero possibile di righe del dataset originario ma eliminando allo stesso tempo i casi clinicamente estremi e rari, in modo da ottenere un dataset più generico, privo di situazioni patologiche estreme.

In [39]:
thresholds = {
    "waistline": 200,
    "sight_left": 4,
    "sight_right": 4,
    "SBP": 240,
    "DBP": 160,
    "BLDS": 600,
    "tot_chole": 1000,
    "HDL_chole": 700,
    "LDL_chole": 2000,
    "triglyceride": 3500,
    "serum_creatinine": 30,
    "SGOT_AST": 2000,
    "SGOT_ALT": 2000,
    "gamma_GTP": 900,
}

for col, threshold in thresholds.items():
    dataset.loc[dataset[col] > threshold, col] = None

# Rimozione delle righe con valori mancanti
dataset_cleaned = dataset.dropna(subset=thresholds.keys())

Ora verranno rieseguite tutte le operazioni di visualizzazione dei dati presenti nel dataset per comprendere la distribuzione di questi dopo l'operazione di rimozione degli outliers.

In [ ]:
# Analisi iniziale del dataset
print("\nPrime righe del dataset:")
print(dataset_cleaned.head())

print("\nDimensioni del dataset:")
print(dataset_cleaned.shape)

print("\nTipi di dati:")
print(dataset_cleaned.info())

print("\nStatistiche descrittive di base:")
print(dataset_cleaned.describe())

In [ ]:
smoke = dataset_cleaned["SMK_stat_type_cd"].value_counts()
drink = dataset_cleaned["DRK_YN"].value_counts()

# Distribuzione SMK_stat_type_cd
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1) # Num righe, num colonne, posizione 
sns.barplot(x=smoke.index, y=smoke.values, palette="Blues_d")
plt.title("Distribuzione delle classi relative al fumo")
plt.xlabel("Tipo di fumatore")
plt.ylabel("Frequenza")

# Distribuzione DRK_YN
plt.subplot(1, 2, 2)
sns.barplot(x=drink.index, y=drink.values, palette="Greens_d")
plt.title("Distribuzione delle classi relative al bere")
plt.xlabel("Bevitore (Y/N)")
plt.ylabel("Frequenza")

plt.tight_layout()
plt.show()

In [ ]:
dataset_mod = dataset_cleaned.drop(columns=["sex", "SMK_stat_type_cd", "DRK_YN"])

# Boxplot per le principali feature numeriche
plt.figure(figsize=(25, 20))
for i, col in enumerate(dataset_mod.columns):
    plt.subplot(5, 5, i + 1)
    sns.boxplot(y=dataset_mod[col], color="skyblue")
    plt.title(col)
    plt.tight_layout()
plt.show()

In [ ]:
# Verifica dei valori massimi e distanza dal secondo massimo e dalla media

for col in dataset_mod.columns:
    max = dataset_mod[col].max()
    max_count = (dataset_mod[col] == max).sum()
    second_max = dataset_mod[col][dataset_mod[col] < max].max()
    distance_max = max - second_max
    mean_value = dataset[col].mean()
    distance_mean = max - mean_value

    print(f"Colonna: {col}")
    print(f"    -Valore massimo: {max}")
    print(f"    -Occorrenze del massimo: {max_count}")
    print(f"    -Secondo massimo: {second_max}")
    print(f"    -Distanza tra massimo e secondo massimo: {distance_max}")
    print(f"    -Media: {mean_value}")
    print(f"    -Distanza tra massimo e media: {distance_mean}")
    print()

In [ ]:
# Determinazione del numero di valori "fuori scala" (outliers) per ogni feature

for col in dataset_mod.columns:
    Q1 = dataset_mod[col].quantile(0.25)
    Q3 = dataset_mod[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = dataset_mod[(dataset_mod[col] < lower_bound) | (dataset_mod[col] > upper_bound)]
    print(f"Colonna: {col}")
    print(f"    -Valori fuori scala: {len(outliers)}")
    print(f"    -Limiti: {lower_bound} - {upper_bound}")
    print()

Una volta terminata la fase di analisi e pulizia dei dati, sarebbe opportuno procedere con le operazioni di encoding e scaling dei dati (visto che sono presenti features con valori molto grandi e altre con valori più piccoli), però pensiamo sia più opportuno fare queste operazioni eventualmente in un secondo momento a seconda dei modelli di ML che decidiamo di utilizzare.
Per ora ci limitiamo a fare l'encoding delle feature categoriali.

In [64]:
categorical_cols = ["sex", "DRK_YN"]

for col in categorical_cols:
    encoder = LabelEncoder()
    dataset_cleaned[col] = encoder.fit_transform(dataset_cleaned[col])


/var/folders/9k/0zc4z1md49d58jhbq4fgqjg40000gn/T/ipykernel_1259/3293868468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cleaned[col] = encoder.fit_transform(dataset_cleaned[col])
/var/folders/9k/0zc4z1md49d58jhbq4fgqjg40000gn/T/ipykernel_1259/3293868468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cleaned[col] = encoder.fit_transform(dataset_cleaned[col])


Di seguito riportiamo un semplice modello di albero decisionale per vedere che lower bound di accuratezza abbiamo

In [90]:
# Divisione del dataset in feature e target (X e Y)
target_smoke = "SMK_stat_type_cd"
target_drink = "DRK_YN"
X_smoke = dataset_cleaned.iloc[:, dataset_cleaned.columns != target_smoke]
#X_smoke = dataset_cleaned.drop(columns=["SMK_stat_type_cd", "DRK_YN"])
Y_smoke = dataset_cleaned[target_smoke]
X_drink = dataset_cleaned.iloc[:, dataset_cleaned.columns != target_drink]
#X_drink = dataset_cleaned.drop(columns=["SMK_stat_type_cd", "DRK_YN"])
Y_drink = dataset_cleaned[target_drink]

In [91]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_smoke, Y_smoke, test_size=0.3, random_state=42)
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train_s, y_train_s)
y_pred = model.predict(X_test_s)
accuracy = accuracy_score(y_test_s, y_pred)
print(f"Accuratezza: {accuracy:.4f}")

Accuratezza: 0.6240


In [92]:
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_drink, Y_drink, test_size=0.3, random_state=42)
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train_d, y_train_d)
y_pred = model.predict(X_test_d)
accuracy = accuracy_score(y_test_d, y_pred)
print(f"Accuratezza: {accuracy:.4f}")

Accuratezza: 0.6434
